In [1]:
#for pretty print to all vars
from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all" #default="last_expr"

#to_display_any_htmls
from IPython.core.display import display, HTML

In [2]:
StorParam=dict()
StorParam['dataset']=dict()
StorParam['splits']=dict()
StorParam['hparams']=dict()
StorParam['metrics']=dict()
StorParam['filesMisClf']=dict()

In [3]:
#Change dir to read dataset files from zip's extract
import os
os.chdir('C:/Users/mihir.parikh/PyNb/wk_data_op/')
files=list()
categories=[fl for fl in os.listdir('.') if os.path.isdir(fl)]
for folder in categories:
        files+=[folder+'/'+p for p in os.listdir(folder)]
print(categories," Total files read:",len(files))

['ADD', 'BUY', 'EPG']  Total files read: 710


In [4]:
#preprocess file's text_contents before laoding to pandas dataframe
import re
dataF={'content':[],'catg':[]}
newfiles=[]
#adding to dataFrame after preprocessing:
#    [remove 1st few not null lines, remove files of less than 10 chars, merge concurrent spaces lines,etc]
skip1stLines=2
for fi in files:
    with open(fi,encoding="utf8") as f:
        #print("for file",fi)
        PREPROC=f.read()
        #make non-space similar to space
        PREPROC=PREPROC.replace(' ',' ') #care...left side wala asli space nai hai
        while PREPROC.count('  '):PREPROC=PREPROC.replace('  ',' ')
        while PREPROC.count('\n '):PREPROC=PREPROC.replace('\n ','\n')
        while PREPROC.count(' \n'):PREPROC=PREPROC.replace(' \n','\n')
        while PREPROC.count('\n\n'):PREPROC=PREPROC.replace('\n\n','\n')
        PREPROC=' '.join(PREPROC.split('\n')[0 if fi.startswith('Others') else skip1stLines:])
        #PREPROC=re.sub(r"\d+(\.\d*)?", r"",PREPROC) #remove all numbers (digits)
        PREPROC=re.sub(r"[(](\S*?)[)]", r"",PREPROC) #remove all b/w (brackets)
        PREPROC=re.sub(r"[^a-zA-Z ]", r"",PREPROC) #remove all non alpha (brackets)
        while PREPROC.count('  '):PREPROC=PREPROC.replace('  ',' ')
        if len(PREPROC)<10:continue
        dataF['content'].append(PREPROC)
        dataF['catg'].append(fi.split('/')[-2])
        newfiles.append(fi)
skippedfiles=list(set(files)-set(newfiles))
print(len(skippedfiles),"files ignored. New Dataset has",len(newfiles),"files.")
##check skip files of some catg
#sorted(list(filter((lambda st:st.startswith('MC')),skippedfiles)))

StorParam['dataset']['files']=files
StorParam['dataset']['newfiles']=newfiles
StorParam['dataset']['preproc_comment']="default"

skippedfiles

4 files ignored. New Dataset has 706 files.


['EPG/45,911.pdf.txt',
 'EPG/171006_Protections.pdf.txt',
 'EPG/45,910.pdf.txt',
 'BUY/reg-level_19CSR-RS30-61-105_The Day Care Provider and Other Day Care Personnel.xml.txt']

In [5]:
#load to pandas dataframe and have a peek on each catg
import pandas as pd
df = pd.DataFrame.from_dict(dataF)
print(df['catg'].value_counts())
for k in df['catg'].value_counts().keys():display(HTML(df[(df['catg']==k)].head(5).to_html()))

StorParam['dataset']['catg_val_count']=df['catg'].value_counts()

EPG    291
ADD    261
BUY    154
Name: catg, dtype: int64


,catg,content
415,EPG,Wasbtngton October MEMORANDUM FOR ALL COMPONEN...
416,EPG,UNITED STATES DISTRICT COURT FOR THE DISTRICT ...
417,EPG,UNITED STATES COURT OF APPEALS FOR THE NINTH C...
418,EPG,UNITED STATES COURT OF APPEALS FOR THE NINTH C...
419,EPG,United States Court of Appeals For the Seventh...


,catg,content
0,ADD,MIDDLE DISTRICT OF FLORIDA TAMPA DIVISION KENN...
1,ADD,UNITED STATES DISTRICT COURT EASTERN DISTRICT...
2,ADD,DISTRICT OF MINNESOTA James Chavira Plaintiff ...
3,ADD,UNITED STATES COURT OF APPEALS TENTH CIRCUIT K...
4,ADD,DISTRICT OF KANSAS Equal Employment Opportunit...


,catg,content
261,BUY,CFR The average unweighted amount and average...
262,BUY,IN THE SUPREME COURT OF PENNSYLVANIA WESTERN D...
263,BUY,IN THE SUPREME COURT OF PENNSYLVANIA WESTERN D...
264,BUY,RELATES TO KRS Chapter STATUTORY AUTHORITY KRS...
265,BUY,January Term FILED June No released at pm RORY...


In [6]:
Randoms=(42,420)
SplitRatios=(.5,.3,.2) #train test cv
#partial import of sklearn. will full load sub-modules as needed.
import sklearn
#split data for training and testing
from sklearn.model_selection import train_test_split
X=df['content']
Y=df['catg']
#SplitRatio_ts=0.4
SplitRatio_ts=sum(SplitRatios[1:])#1-SplitRatios[0]
#SplitRatio_cv=0.5
SplitRatio_cv=SplitRatios[-1]/SplitRatio_ts
Random_ts,Random_cv=Randoms
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=SplitRatio_ts,random_state=Random_ts)
X_test,X_cv,Y_test,Y_cv=sklearn.model_selection.train_test_split(X_test,Y_test,test_size=SplitRatio_cv,random_state=Random_cv)
print("Train",[X_train.shape,Y_train.shape],"Test",[X_test.shape,Y_test.shape],"CV",[X_cv.shape,Y_cv.shape])
#now data is ready
StorParam['splits']['Ratios']=SplitRatios
StorParam['splits']['Randoms']=Randoms

Train [(353,), (353,)] Test [(211,), (211,)] CV [(142,), (142,)]


### New method One-shot func: fitPipe

In [7]:
#pipelining
from sklearn.pipeline import Pipeline
def fitPipe(algoname,text_clf_):
    global StorParam,X_train,X_test,Y_train,Y_test,X_cv,Y_cv
    StorParam['hparams'][algoname]={x:text_clf_.named_steps[x].get_params() for x in text_clf_.named_steps.keys()}
    #fit_transforms of data to pipeline
    fit_=text_clf_.fit(X_train,Y_train)

    #testing model
    print("--> Accuracy")
    
    TRpredicted_ = text_clf_.predict(X_train)
    #trscr_=text_clf_.score(X_train,Y_train)
    crtab_tr=pd.crosstab(Y_train,TRpredicted_,rownames=["True"],colnames=["Train_Predicted"],margins=True)
    trscr_=1-(crtab_tr.All.All-sum([crtab_tr[c][c] for c in text_clf_.classes_]))/crtab_tr.All.All
    print("TRAIN",trscr_)

    TSpredicted_ = text_clf_.predict(X_test)
    #tsscr_=text_clf_.score(X_test,Y_test)
    crtab_ts=pd.crosstab(Y_test,TSpredicted_,rownames=["True"],colnames=["Test_Predicted"],margins=True)
    tsscr_=1-(crtab_ts.All.All-sum([crtab_ts[c][c] for c in text_clf_.classes_]))/crtab_ts.All.All
    print("TEST",tsscr_)
    
    StorParam['metrics'][algoname]={'train':trscr_,'test':tsscr_,'cv':None}
    StorParam['filesMisClf'][algoname]={'train':list(),'test':list(),'cv':list()}

    #misClassifications
    print("--> Train MisClassification")
    misClTr_=list()
    #sklearn.metrics.confusion_matrix
    display(HTML(crtab_tr.to_html()))
    for i in range(len(TRpredicted_)):
        if TRpredicted_[i]!=Y_train.iloc[i]:
            print("X_Train.values[",i,"] Predicted",TRpredicted_[i],"Actual",Y_train.iloc[i],
                  "File:[errI="+str(X_train.index[i]),newfiles[X_train.index[i]],"]")
            misClTr_.append(i)
            StorParam['filesMisClf'][algoname]['train'].append(newfiles[X_train.index[i]])
    print(len(misClTr_)," misses in TrainData")

    print("--> Test MisClassification")
    misClTs_=list()
    #sklearn.metrics.confusion_matrix
    display(HTML(crtab_ts.to_html()))
    for i in range(len(TSpredicted_)):
        if TSpredicted_[i]!=Y_test.iloc[i]:
            print("X_test.values[",i,"] Predicted",TSpredicted_[i],"Actual",Y_test.iloc[i],
                  "File:[errI="+str(X_test.index[i]),newfiles[X_test.index[i]],"]")
            misClTs_.append(i)
            StorParam['filesMisClf'][algoname]['test'].append(newfiles[X_test.index[i]])

    print(len(misClTs_)," misses in TestData")
    
    #push these vars to global namespace
    returnD={k+algoname:v for k,v in locals().items() if k.endswith('_')}
    for k,v in returnD.items():globals()[k]=v

## Multinomial NAIVE BAYES

In [8]:
algoname='nb'
#naive_bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

text_clf_nb = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,3))),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB(alpha=0.01))
                    ])
fitPipe('nb',text_clf_nb)

--> Accuracy
TRAIN 1.0
TEST 0.8957345971563981
--> Train MisClassification


Train_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,120,0,0,120
BUY,0,69,0,69
EPG,0,0,164,164
All,120,69,164,353


0  misses in TrainData
--> Test MisClassification


Test_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,82,0,7,89
BUY,0,41,8,49
EPG,7,0,66,73
All,89,41,81,211


X_test.values[ 13 ] Predicted EPG Actual ADD File:[errI=149 ADD/17-120.pdf.txt ]
X_test.values[ 31 ] Predicted ADD Actual EPG File:[errI=587 EPG/45752.pdf.txt ]
X_test.values[ 37 ] Predicted EPG Actual BUY File:[errI=382 BUY/RocheleauElder.pdf.txt ]
X_test.values[ 45 ] Predicted ADD Actual EPG File:[errI=692 EPG/45868.pdf.txt ]
X_test.values[ 46 ] Predicted EPG Actual ADD File:[errI=179 ADD/17-150.pdf.txt ]
X_test.values[ 58 ] Predicted EPG Actual BUY File:[errI=284 BUY/BuenoLR.pdf.txt ]
X_test.values[ 59 ] Predicted EPG Actual ADD File:[errI=133 ADD/17-103.pdf.txt ]
X_test.values[ 67 ] Predicted EPG Actual ADD File:[errI=78 ADD/17-043.pdf.txt ]
X_test.values[ 95 ] Predicted EPG Actual ADD File:[errI=163 ADD/17-134.pdf.txt ]
X_test.values[ 96 ] Predicted EPG Actual BUY File:[errI=338 BUY/MaddenMidland.pdf.txt ]
X_test.values[ 101 ] Predicted EPG Actual ADD File:[errI=136 ADD/17-106.pdf.txt ]
X_test.values[ 104 ] Predicted ADD Actual EPG File:[errI=674 EPG/45849.pdf.txt ]
X_test.values[

### SVM SVC [C-Support Vector Classification]

In [9]:
#pipelining
from sklearn.pipeline import Pipeline
#svm_svc
from sklearn.svm import SVC

text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english',ngram_range=(1,3), )),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SVC(decision_function_shape="ovo", C = 1000000, kernel='rbf', gamma ='auto'))
                    ])
fitPipe('svm',text_clf_svm)

--> Accuracy
TRAIN 1.0
TEST 0.933649289099526
--> Train MisClassification


Train_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,120,0,0,120
BUY,0,69,0,69
EPG,0,0,164,164
All,120,69,164,353


0  misses in TrainData
--> Test MisClassification


Test_Predicted,ADD,BUY,EPG,All
True,,,,
ADD,84,0,5,89
BUY,0,44,5,49
EPG,4,0,69,73
All,88,44,79,211


X_test.values[ 13 ] Predicted EPG Actual ADD File:[errI=149 ADD/17-120.pdf.txt ]
X_test.values[ 31 ] Predicted ADD Actual EPG File:[errI=587 EPG/45752.pdf.txt ]
X_test.values[ 37 ] Predicted EPG Actual BUY File:[errI=382 BUY/RocheleauElder.pdf.txt ]
X_test.values[ 45 ] Predicted ADD Actual EPG File:[errI=692 EPG/45868.pdf.txt ]
X_test.values[ 46 ] Predicted EPG Actual ADD File:[errI=179 ADD/17-150.pdf.txt ]
X_test.values[ 58 ] Predicted EPG Actual BUY File:[errI=284 BUY/BuenoLR.pdf.txt ]
X_test.values[ 95 ] Predicted EPG Actual ADD File:[errI=163 ADD/17-134.pdf.txt ]
X_test.values[ 127 ] Predicted EPG Actual BUY File:[errI=319 BUY/GraySeterus.pdf.txt ]
X_test.values[ 134 ] Predicted EPG Actual BUY File:[errI=344 BUY/McMahonLVNV.pdf.txt ]
X_test.values[ 152 ] Predicted EPG Actual ADD File:[errI=247 ADD/17-218.pdf.txt ]
X_test.values[ 174 ] Predicted ADD Actual EPG File:[errI=428 EPG/45,882.pdf.txt ]
X_test.values[ 178 ] Predicted EPG Actual ADD File:[errI=108 ADD/17-073.pdf.txt ]
X_test

In [10]:
StorParam.keys()

dict_keys(['dataset', 'splits', 'hparams', 'metrics', 'filesMisClf'])

In [11]:
StorParam['dataset'].keys()

dict_keys(['files', 'newfiles', 'preproc_comment', 'catg_val_count'])

## grid

In [116]:
#grid
import numpy as np
from sklearn.model_selection import GridSearchCV
#import sklearn.base
class Echo(sklearn.base.BaseEstimator,sklearn.base.TransformerMixin):
    def transform(self, X, *_):return X
    def fit(self, *_):return self

pipe = Pipeline([
    ('vect', CountVectorizer(stop_words='english',ngram_range=(1,3))),
    ('TFidf', TfidfTransformer()),
    ('clf', MultinomialNB())                   
])
clfs=[
      { 'clf': [SVC(decision_function_shape="ovo", C = 1000000, kernel='rbf', gamma ='auto')],
        'clf__kernel':['rbf','poly','sigmoid'],
        'clf__C':[10000,1000000,100000,1,10,0.1],
        'clf__degree':[5]
      },
#      { 'clf': [MultinomialNB(alpha=0.01)],
#        'clf__alpha':[0.001,.01,.1,1,10]
#      }
     ]
param_grid = [
    {
        #'vect__ngram_range': sum([[(i,j) for j in range(i,4+1)] for i in range(1,3+1)],[]),
        'vect__ngram_range': [(1,2),(1,3),(2,3)],
        'TFidf': [TfidfTransformer()     ,None],#,Echo()],
        #'clf': [MultinomialNB(alpha=0.01),SVC(decision_function_shape="ovo", C = 1000000, kernel='rbf', gamma ='auto')],
        #'clf__kernel':['rbf','poly'],
        #'clf__alpha':np.arange(1.,2.,0.1)
        **clf
    }
for clf in clfs]

grid = GridSearchCV(pipe, cv=3, n_jobs=2, param_grid=param_grid,error_score=0.0,verbose=2)
param_grid

[{'TFidf': [TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True),
   None],
  'clf': [SVC(C=1000000, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
     max_iter=-1, probability=False, random_state=None, shrinking=True,
     tol=0.001, verbose=False)],
  'clf__C': [10000, 1000000, 100000, 1, 10, 0.1],
  'clf__degree': [5],
  'clf__kernel': ['rbf', 'poly', 'sigmoid'],
  'vect__ngram_range': [(1, 2), (1, 3), (2, 3)]}]

In [117]:
grid.fit(['abr ']*30,[1,2,3]*10)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


[Parallel(n_jobs=2)]: Done 324 out of 324 | elapsed:    7.1s finished


GridSearchCV(cv=3, error_score=0.0,
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words='english',
        ...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid=True, n_jobs=2,
       param_grid=[{'vect__ngram_range': [(1, 2), (1, 3), (2, 3)], 'TFidf': [TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True), None], 'clf': [SVC(C=10000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=5, gamma='auto', kernel='rbf',
  max...l': ['rbf', 'poly', 'sigmoid'], 'clf__C': [10000, 1000000, 100000, 1, 10, 0.1], 'clf__degree': [5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_s

In [118]:
grid.fit(X_train,Y_train)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:  7.1min
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed: 30.0min
[Parallel(n_jobs=2)]: Done 324 out of 324 | elapsed: 59.4min finished


GridSearchCV(cv=3, error_score=0.0,
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words='english',
        ...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid=True, n_jobs=2,
       param_grid=[{'vect__ngram_range': [(1, 2), (1, 3), (2, 3)], 'TFidf': [TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True), None], 'clf': [SVC(C=1000000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=5, gamma='auto', kernel='rbf',
  m...l': ['rbf', 'poly', 'sigmoid'], 'clf__C': [10000, 1000000, 100000, 1, 10, 0.1], 'clf__degree': [5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_s

In [122]:
grid.fit(X,Y)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed: 19.4min
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed: 81.1min
[Parallel(n_jobs=2)]: Done 324 out of 324 | elapsed: 162.2min finished


GridSearchCV(cv=3, error_score=0.0,
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words='english',
        ...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid=True, n_jobs=2,
       param_grid=[{'vect__ngram_range': [(1, 2), (1, 3), (2, 3)], 'TFidf': [TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True), None], 'clf': [SVC(C=1000000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=5, gamma='auto', kernel='rbf',
  m...l': ['rbf', 'poly', 'sigmoid'], 'clf__C': [10000, 1000000, 100000, 1, 10, 0.1], 'clf__degree': [5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_s

In [123]:
grid.best_score_

0.9362606232294618

In [124]:
grid.best_estimator_.score(X_test,Y_test)

0.985781990521327

In [125]:
grid.best_estimator_.score(X_cv,Y_cv)

1.0

In [126]:
grid.best_params_

{'TFidf': TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True),
 'clf': SVC(C=1000000, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovo', degree=5, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
 'clf__C': 1000000,
 'clf__degree': 5,
 'clf__kernel': 'rbf',
 'vect__ngram_range': (1, 2)}

In [127]:
grid.cv_results_['params'].__len__()

108

In [46]:
grid.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_TFidf', 'param_clf', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'mean_train_score', 'std_train_score'])

In [112]:
grid.cv_results_['mean_fit_time']

array([14.12135839, 13.513762  , 14.57875641, 13.30735342, 14.10935442,
       13.37135347, 13.33268698, 13.35768723, 13.40902114, 13.36135324,
       13.5196871 , 13.35501981, 12.28135149, 13.11401947, 12.27368561,
       13.10768692, 12.26501942, 12.91701905, 13.29835447, 12.7853535 ,
       12.48135217, 13.2270201 , 13.08601936, 12.91111286, 10.84277113,
       10.8367637 , 10.90614883, 10.74138546, 11.21613844, 11.01750867,
       11.73276567, 11.55197191, 11.449531  , 11.26563096])

In [103]:
grid.get_params('param_grid')

{'cv': 3, 'error_score': 0.0, 'estimator': Pipeline(memory=None,
      steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 3), preprocessor=None, stop_words='english',
         ...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]), 'estimator__TFidf': TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True), 'estimator__TFidf__norm': 'l2', 'estimator__TFidf__smooth_idf': True, 'estimator__TFidf__sublinear_tf': False, 'estimator__TFidf__use_idf': True, 'estimator__clf': MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True), 'estimator__clf__alpha': 1.0, 'estimator__clf__class_prior': None, 'estimator__clf__fit_prior': True, 'estimator__memory': None, 'estimator__steps': [('vect',
   CountVectorizer(analyzer='word', 

In [134]:
def approxPIsquared(float_error):
	denom=1
	mypi=8.0/(denom*denom)
	oldpi=0
	while abs(mypi-oldpi)>=float_error:
		oldpi=mypi
		mypi=mypi+8.0/(denom*denom)
	return mypi


In [ ]:
approxPIsquared(0.0001)